## minTree
This program uses minTree filtering for grain extraction.

Previous = (Core extraction with min-tree.          This program uses min-tree filtering for core extraction. Filtering parameters shoudl automatically adapt with most of the EFR greyscale subresolutions, but feel free to experiment with other values if needed.)

## More information

- argparse : to manage command line arguments.
- os : for manipulating file paths.
- tifffile : for reading and writing TIFF files.
- numpy (alias np) : for manipulating numerical vectors.
- timeit for timer() : to measure execution time.
- gc : for manual memory management via the garbage collector.
- hg : an image processing library (a hierarchical processing library).

#### French

- argparse : pour gérer les arguments de la ligne de commande.
- os : pour manipuler les chemins de fichiers.
- tifffile : pour lire et écrire des fichiers TIFF.
- numpy (alias np) : pour manipuler les tableaux numériques.
- timeit pour le timer() : pour mesurer le temps d'exécution.
- gc : pour la gestion manuelle de la mémoire via le ramasse-miettes.
- hg : une bibliothèque de traitement d'images (une bibliothèque de traitement par hiérarchies).

# Code

In [ ]:
__author__ = "Lysandre Macke"
# Adapted by Jonathan Palisse
__contact__ = "lmacke@unistra.com"

### Imports

In [9]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    import higra as hg # ignore the generated warnings

import argparse
import numpy as np
import skimage.morphology as morph
from skimage.measure import label
import tifffile # tiff file manipulation
import os
from dstyle import *
from timeit import default_timer as timer
import gc

### Command-line program management

Command example : \
python minTree/minTree.py image.tif 6\
python minTree/minTree --help

In [ ]:
parser = argparse.ArgumentParser(
                    prog = "minTree",
                    description = "Core extraction with minTree.")

parser.add_argument("file", help="The path to the tiff file.")
parser.add_argument("adjacency", help="Adjacency of the mask (6 or 26)")

args = parser.parse_args()
filepath = args.file
filename = os.path.splitext(os.path.basename(filepath))[0]
adjacency = args.adjacency

### Initialise tools and display animation

In [ ]:
timestamp_0 = timer()
gc.disable()
animation:TermLoading = TermLoading()
animation.show("Processing " + filename,
    finish_message = "\x1b[2K-- Binarization done.",
    failed_message = "\x1b[2K-- Error occured while processing data.")

#### TIFF image loading and 8-bit conversion

In [ ]:
with tifffile.TiffFile(filepath) as tif:
    image = np.array([page.asarray() for page in tif.pages])

image = (image/256).astype("uint8")

#### Creating the result image and configuring the graph

neighbours is a formal representation of the connections between pixels \
https://higra.readthedocs.io/en/stable/python/graph_image.html

In [ ]:
res = np.zeros_like(image)

if (adjacency == 26):
    # 26 adjacency implicit graph
    mask = [[[1, 1, 1], [1, 1, 1], [1, 1, 1]],
            [[1, 1, 1], [1, 0, 1], [1, 1, 1]],
            [[1, 1, 1], [1, 1, 1], [1, 1, 1]]]
else:
    # 6 adjacency implicit graph
    mask = [[[0, 0, 0], [0, 1, 0], [0, 0, 0]],
            [[0, 1, 0], [1, 0, 1], [0, 1, 0]],
            [[0, 0, 0], [0, 1, 0], [0, 0, 0]]]

neighbours = hg.mask_2_neighbours(mask)
graph = hg.get_nd_regular_implicit_graph(image.shape, neighbours)

#### Building the minimal tree and calculating attributes

In [ ]:
# construct min tree from image
tree, altitudes = hg.component_tree_min_tree(graph, image)

# compute attributes
area = hg.attribute_area(tree)
height = hg.attribute_height(tree, altitudes)

depth = hg.attribute_depth(tree)

#### Tree simplification

You can change the values in np.logical_or()

In [ ]:
max_height = np.max(height)
min_area   = np.min(area)
avg_area   = np.average(area)

# remove unwanted nodes
# the nodes we want to keep have a strong value and a small area
# unwanted_nodes = np.logical_or(height < 0.16*max_height, area > 0.65*avg_area)
unwanted_nodes = np.logical_or(height < 0.14*max_height,   area > avg_area)

tree, node = hg.simplify_tree(tree, unwanted_nodes)
new_altitudes = altitudes[node]

#### Image reconstruction

In [ ]:
# reconstruct tree
res = hg.reconstruct_leaf_data(tree, new_altitudes)
res = (res < np.max(res)).astype(int)*255

output_name = filename + "_minTree_segment_raw.tif"

animation.finished = True
animation.show("Saving result to " + output_name,
    finish_message = "\x1b[2K-- Generated " + output_name + " successfully " + \
                        "(time : %.2f s)" % (timer() - timestamp_0),
    failed_message = "\x1b[2K-- Error occured while saving result.")

#### Saving the TIFF file and cleaning

Ici ça lance Gmic, faut changer parce que on n'a pas ce qu'il faut en terme de RAM

In [ ]:
# write out file...
tifffile.imwrite(output_name, res)

animation.finished = True
os.system("gmic " + output_name + " a z")       #Problem
gc.enable()
exit(0)